# Graph

In [1]:
import tensorflow as tf

상수를 더하기 예제

In [7]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)

total = a + b

# tensorboard를 위해 그래프 추가
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

with tf.Session() as sess:
    print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


그래프의 동작 방식을 이해하기 위한 예제이다. 다음은 그래프가 3번 실행 된다.
마지막 실행에서 out1, out2가 동일한 vec값을 사용했다는 점을 확인하기 바란다.

In [10]:
#변수 3개 반환
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2

with tf.Session() as sess:
    print('vec: {}'.format(sess.run(vec)))
    print('vec: {}'.format(sess.run(vec)))
    # out1, out2가 동일한 vec을 사용했다
    print('out: {}'.format(sess.run((out1, out2))))

vec: [ 0.56069469  0.87755346  0.88185894]
vec: [ 0.09359825  0.5918963   0.2976625 ]
out: (array([ 1.70737994,  1.05672693,  1.97715557], dtype=float32), array([ 2.70737982,  2.05672693,  2.97715569], dtype=float32))


입력 값을 넣기 위해 placeholder를 추가한 내용이다.

In [13]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

z = x + y
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x: 3, y: 4.5}))
    print(sess.run(z, feed_dict={x: [1, 3], y:[2, 4]}))

7.5
[ 3.  7.]


In [ ]:
my_data = np.array(range(10))